## DC2 Generate Cat and Split Data 

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from os import environ
from pathlib import Path

import GCRCatalogs
import pandas as pd
import torch
from hydra import compose, initialize
from hydra.utils import instantiate
import pickle

from bliss.catalog import SourceType
from bliss.surveys.dc2 import DC2DataModule, wcs_from_wcs_header_str

# Generate LSST data for Regressor

In [4]:
output_dir = Path("/data/scratch/qiaozhih/data/redshift/dc2/")
output_dir.mkdir(parents=True, exist_ok=True)

In [4]:
import GCRCatalogs
import pandas as pd
import torch
from bliss.catalog import FullCatalog, SourceType

lsst_root_dir = "/data/scratch/dc2_nfs/"
GCRCatalogs.set_root_dir(lsst_root_dir)
lsst_catalog_gcr = GCRCatalogs.load_catalog("desc_dc2_run2.2i_dr6_object_with_truth_match")

lsst_catalog_sub = lsst_catalog_gcr.get_quantities(
    [
        "id_truth",
        "mag_u_cModel",
        "mag_g_cModel",
        "mag_r_cModel",
        "mag_i_cModel",
        "mag_z_cModel",
        "mag_y_cModel",
        "magerr_u_cModel",
        "magerr_g_cModel",
        "magerr_r_cModel",
        "magerr_i_cModel",
        "magerr_z_cModel",
        "magerr_y_cModel",
    ]
)
lsst_catalog_df = pd.DataFrame(lsst_catalog_sub)



In [5]:
lsst_catalog_df.describe()

,mag_y_cModel,magerr_i_cModel,magerr_z_cModel,mag_i_cModel,mag_u_cModel,mag_z_cModel,id_truth,magerr_u_cModel,magerr_y_cModel,magerr_g_cModel,mag_g_cModel,mag_r_cModel,magerr_r_cModel
count,9.987282e+06,1.101893e+07,1.079363e+07,1.093483e+07,9.649256e+06,1.029231e+07,1.111621e+07,1.057578e+07,1.071678e+07,1.101304e+07,1.094410e+07,1.101770e+07,1.103876e+07
mean,inf,NaN,NaN,inf,inf,inf,8.088583e+09,inf,NaN,NaN,inf,inf,inf
std,NaN,NaN,NaN,NaN,NaN,NaN,4.274614e+10,NaN,NaN,NaN,NaN,NaN,NaN
min,1.143206e+01,-inf,-inf,1.189937e+01,1.399609e+01,1.159003e+01,6.763860e+05,-4.706063e+08,-inf,-inf,1.291604e+01,1.227066e+01,-9.468324e+11
25%,2.392812e+01,6.500480e-02,1.057525e-01,2.467404e+01,2.561706e+01,2.423471e+01,6.825068e+09,1.311172e-01,1.292641e-01,6.841042e-02,2.551972e+01,2.516218e+01,5.814535e-02
50%,2.481313e+01,1.479706e-01,2.820019e-01,2.561176e+01,2.647421e+01,2.521701e+01,7.750733e+09,3.612945e-01,3.468699e-01,1.426473e-01,2.635767e+01,2.603854e+01,1.232727e-01
75%,2.562102e+01,2.613201e-01,5.848774e-01,2.627653e+01,2.732608e+01,2.603465e+01,7.878704e+09,7.734282e-01,7.187968e-01,2.248982e-01,2.693405e+01,2.661246e+01,1.935008e-01
max,inf,inf,inf,inf,inf,inf,9.026235e+12,inf,inf,inf,inf,inf,inf


In [6]:
import numpy as np
lsst_catalog_df_na = lsst_catalog_df.replace([np.inf, -np.inf], np.nan)

In [7]:
lsst_catalog_df_nona = lsst_catalog_df_na.dropna()

In [8]:
lsst_catalog_df_nona.describe()

,mag_y_cModel,magerr_i_cModel,magerr_z_cModel,mag_i_cModel,mag_u_cModel,mag_z_cModel,id_truth,magerr_u_cModel,magerr_y_cModel,magerr_g_cModel,mag_g_cModel,mag_r_cModel,magerr_r_cModel
count,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06
mean,2.449080e+01,4.641381e+01,3.376254e+03,2.501757e+01,2.627893e+01,2.479081e+01,8.062859e+09,5.928891e+03,1.387315e+02,1.956296e+02,2.583652e+01,2.540720e+01,8.462013e+02
std,1.725277e+00,6.580380e+04,9.325938e+06,1.616828e+00,1.637081e+00,1.772456e+00,3.906263e+10,1.498706e+07,2.292829e+05,2.871290e+05,1.464796e+00,1.491504e+00,1.101339e+06
min,1.143206e+01,2.846753e-05,4.018415e-05,1.189937e+01,1.437855e+01,1.159003e+01,8.115470e+05,2.694765e-04,4.407217e-05,5.204819e-05,1.291604e+01,1.227066e+01,2.749518e-05
25%,2.368605e+01,4.860332e-02,1.059764e-01,2.427482e+01,2.538903e+01,2.392407e+01,6.820354e+09,1.747186e-01,1.545336e-01,5.188620e-02,2.514867e+01,2.475342e+01,4.200911e-02
50%,2.463077e+01,1.146499e-01,2.538098e-01,2.529204e+01,2.629794e+01,2.497002e+01,7.750373e+09,3.796795e-01,3.414059e-01,1.101115e-01,2.603747e+01,2.570830e+01,9.313366e-02
75%,2.540578e+01,2.043973e-01,5.142233e-01,2.600035e+01,2.712148e+01,2.578308e+01,7.878367e+09,7.866697e-01,6.780241e-01,1.926910e-01,2.671187e+01,2.636522e+01,1.624493e-01
max,4.732455e+01,1.764284e+08,2.586460e+10,4.754039e+01,5.267220e+01,5.222029e+01,9.025765e+12,4.138183e+10,5.900637e+08,7.304515e+08,5.016285e+01,5.092666e+01,2.770628e+09


In [9]:
new_name = {
    "id_truth": "id",
    "mag_u_cModel": "mag_u_lsst",
    "mag_g_cModel": "mag_g_lsst",
    "mag_r_cModel": "mag_r_lsst",
    "mag_i_cModel": "mag_i_lsst",
    "mag_z_cModel": "mag_z_lsst",
    "mag_y_cModel": "mag_y_lsst",
    "magerr_u_cModel": "mag_err_u_lsst",
    "magerr_g_cModel": "mag_err_g_lsst",
    "magerr_r_cModel": "mag_err_r_lsst",
    "magerr_i_cModel": "mag_err_i_lsst",
    "magerr_z_cModel": "mag_err_z_lsst",
    "magerr_y_cModel": "mag_err_y_lsst",
}

In [10]:
lsst_catalog_df_nona_newname = lsst_catalog_df_nona.rename(new_name, axis=1)

In [50]:
lsst_catalog_df_nona_newname[-1-100_000: -1].to_pickle("/data/scratch/qiaozhih/data/redshift/dc2/lsst_val_nona_100k.pkl")
lsst_catalog_df_nona_newname[: 200_000].to_pickle("/data/scratch/qiaozhih/data/redshift/dc2/lsst_train_nona_200k.pkl")

In [11]:
truth_catalog_pickle_file = output_dir / "truth_catalog.pkl"
GCRCatalogs.set_root_dir("/nfs/turbo/lsa-regier/lsstdesc-public/dc2")
if truth_catalog_pickle_file.exists():
    with open(truth_catalog_pickle_file, "rb") as inputp: 
        truth_cat_data = pickle.load(inputp)
else:   
    truth_cat = GCRCatalogs.load_catalog("desc_dc2_run2.2i_dr6_truth") 
    print(sorted(truth_cat.list_all_quantities()))
    truth_cat_data = truth_cat.get_quantities([
        "id", "redshift",
    ])
    with open(truth_catalog_pickle_file, "wb") as outp:
        pickle.dump(truth_cat_data, outp, pickle.HIGHEST_PROTOCOL)

truth_cat_df = pd.DataFrame(truth_cat_data)


In [12]:
lsst_truth_redshift_df = truth_cat_df.merge(
    lsst_catalog_df_nona_newname, 
    left_on="id", 
    right_on="id", 
    how="left" 
)

In [13]:
lsst_truth_redshift_df_nona = lsst_truth_redshift_df.dropna()

In [16]:
lsst_truth_redshift_df_nona = lsst_truth_redshift_df_nona.drop("match_objectId", axis=1)

In [17]:
lsst_truth_redshift_df_nona.to_pickle("/data/scratch/qiaozhih/data/redshift/dc2/lsst_truth_redshift_nona.pkl")

In [18]:
lsst_truth_redshift_df_nona.describe()

,redshift,id,mag_y_lsst,mag_err_i_lsst,mag_err_z_lsst,mag_i_lsst,mag_u_lsst,mag_z_lsst,mag_err_u_lsst,mag_err_y_lsst,mag_err_g_lsst,mag_g_lsst,mag_r_lsst,mag_err_r_lsst
count,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06,7.691795e+06
mean,1.136858e+00,8.062859e+09,2.449080e+01,4.641381e+01,3.376254e+03,2.501757e+01,2.627893e+01,2.479081e+01,5.928891e+03,1.387315e+02,1.956296e+02,2.583652e+01,2.540720e+01,8.462013e+02
std,6.071841e-01,3.906263e+10,1.725277e+00,6.580380e+04,9.325938e+06,1.616828e+00,1.637081e+00,1.772456e+00,1.498706e+07,2.292829e+05,2.871290e+05,1.464796e+00,1.491504e+00,1.101339e+06
min,0.000000e+00,8.115470e+05,1.143206e+01,2.846753e-05,4.018415e-05,1.189937e+01,1.437855e+01,1.159003e+01,2.694765e-04,4.407217e-05,5.204819e-05,1.291604e+01,1.227066e+01,2.749518e-05
25%,7.289815e-01,6.820354e+09,2.368605e+01,4.860332e-02,1.059764e-01,2.427482e+01,2.538903e+01,2.392407e+01,1.747186e-01,1.545336e-01,5.188620e-02,2.514867e+01,2.475342e+01,4.200911e-02
50%,1.071423e+00,7.750373e+09,2.463077e+01,1.146499e-01,2.538098e-01,2.529204e+01,2.629794e+01,2.497002e+01,3.796795e-01,3.414059e-01,1.101115e-01,2.603747e+01,2.570830e+01,9.313366e-02
75%,1.484574e+00,7.878367e+09,2.540578e+01,2.043973e-01,5.142233e-01,2.600035e+01,2.712148e+01,2.578308e+01,7.866697e-01,6.780241e-01,1.926910e-01,2.671187e+01,2.636522e+01,1.624493e-01
max,3.072735e+00,9.025765e+12,4.732455e+01,1.764284e+08,2.586460e+10,4.754039e+01,5.267220e+01,5.222029e+01,4.138183e+10,5.900637e+08,7.304515e+08,5.016285e+01,5.092666e+01,2.770628e+09


In [19]:
lsst_truth_redshift_df_nona[:200_000].to_pickle("/data/scratch/qiaozhih/data/redshift/dc2/lsst_truth_redshift_train_nona_200k.pkl")
lsst_truth_redshift_df_nona[-1-100_000:-1].to_pickle("/data/scratch/qiaozhih/data/redshift/dc2/lsst_truth_redshift_val_nona_100k.pkl")

# Generate Cat data

In [2]:
output_dir = Path("./DC2_generate_catalog_output/")
output_dir.mkdir(parents=True, exist_ok=True)

In [8]:
truth_catalog_pickle_file = output_dir / "truth_catalog.pkl"
GCRCatalogs.set_root_dir("/nfs/turbo/lsa-regier/lsstdesc-public/dc2")
if truth_catalog_pickle_file.exists():
    with open(truth_catalog_pickle_file, "rb") as inputp: 
        truth_cat_data = pickle.load(inputp)
else:   
    truth_cat = GCRCatalogs.load_catalog("desc_dc2_run2.2i_dr6_truth") 
    print(sorted(truth_cat.list_all_quantities()))
    truth_cat_data = truth_cat.get_quantities([
        "id", "match_objectId", "cosmodc2_id", "ra", "dec", "truth_type", 
        "flux_g", "flux_i", "flux_r", "flux_u", "flux_y", "flux_z",
        "redshift",
    ])
    with open(truth_catalog_pickle_file, "wb") as outp:
        pickle.dump(truth_cat_data, outp, pickle.HIGHEST_PROTOCOL)

truth_cat_df = pd.DataFrame(truth_cat_data)
truth_cat_df.rename(columns={"redshift": "redshifts"}, inplace=True)

In [11]:
galaxy_params_pickle_file = output_dir / "galaxy_params.pkl"
if galaxy_params_pickle_file.exists():
    with open(galaxy_params_pickle_file, "rb") as inputp: 
        galaxy_params_data = pickle.load(inputp)
else:
    GCRCatalogs.set_root_dir("/nfs/turbo/lsa-regier")
    galaxy_params = GCRCatalogs.load_catalog("desc_cosmodc2")
    print(sorted(galaxy_params.list_all_quantities()))
    galaxy_params_data = galaxy_params.get_quantities([
        "galaxy_id",
        "shear_1", "shear_2",
        "ellipticity_1_true", "ellipticity_2_true",
    ])
    with open(galaxy_params_pickle_file, "wb") as outp:
        pickle.dump(galaxy_params_data, outp, pickle.HIGHEST_PROTOCOL)

galaxy_params_df = pd.DataFrame(galaxy_params_data)

['A_v', 'A_v_bulge', 'A_v_disk', 'Mag_true_Y_lsst_z0', 'Mag_true_Y_lsst_z0_no_host_extinction', 'Mag_true_g_lsst_z0', 'Mag_true_g_lsst_z0_no_host_extinction', 'Mag_true_g_sdss_z0', 'Mag_true_g_sdss_z0_no_host_extinction', 'Mag_true_i_lsst_z0', 'Mag_true_i_lsst_z0_no_host_extinction', 'Mag_true_i_sdss_z0', 'Mag_true_i_sdss_z0_no_host_extinction', 'Mag_true_r_lsst_z0', 'Mag_true_r_lsst_z0_no_host_extinction', 'Mag_true_r_sdss_z0', 'Mag_true_r_sdss_z0_no_host_extinction', 'Mag_true_u_lsst_z0', 'Mag_true_u_lsst_z0_no_host_extinction', 'Mag_true_u_sdss_z0', 'Mag_true_u_sdss_z0_no_host_extinction', 'Mag_true_y_lsst_z0', 'Mag_true_y_lsst_z0_no_host_extinction', 'Mag_true_z_lsst_z0', 'Mag_true_z_lsst_z0_no_host_extinction', 'Mag_true_z_sdss_z0', 'Mag_true_z_sdss_z0_no_host_extinction', 'R_v', 'R_v_bulge', 'R_v_disk', 'bulge_to_total_ratio_i', 'convergence', 'dec', 'dec_true', 'ellipticity_1_bulge_true', 'ellipticity_1_bulge_true_dc2', 'ellipticity_1_disk_true', 'ellipticity_1_disk_true_dc2', '

In [22]:
psf_params_pickle_file = output_dir / "psf_params.pkl"
if psf_params_pickle_file.exists():
    with open(psf_params_pickle_file, "rb") as inputp: 
        psf_params_data = pickle.load(inputp)
else:
    psf_params = GCRCatalogs.load_catalog("desc_dc2_run2.2i_dr6_object_with_truth_match")
    psf_params_data = psf_params.get_quantities([
        "objectId", "blendedness", "IxxPSF_pixel_g", "IxxPSF_pixel_z", 
        "IxxPSF_pixel_r", "IxxPSF_pixel_i", "IxxPSF_pixel_u", 
        "IxxPSF_pixel_y", "IyyPSF_pixel_g", "IyyPSF_pixel_z", 
        "IyyPSF_pixel_r", "IyyPSF_pixel_i", "IyyPSF_pixel_u", 
        "IyyPSF_pixel_y", "IxyPSF_pixel_g", "IxyPSF_pixel_z", 
        "IxyPSF_pixel_r", "IxyPSF_pixel_i", "IxyPSF_pixel_u", 
        "IxyPSF_pixel_y", "psf_fwhm_g", "psf_fwhm_z", "psf_fwhm_r",
        "psf_fwhm_i", "psf_fwhm_u", "psf_fwhm_y"
    ])
    with open(psf_params_pickle_file, "wb") as outp:
        pickle.dump(psf_params_data, outp, pickle.HIGHEST_PROTOCOL)

### Merge Catalog

In [7]:
truth_cat_df = pd.DataFrame(truth_cat_data)
galaxy_params_df = pd.DataFrame(galaxy_params_data)
psf_params_df = pd.DataFrame(psf_params_data)
truth_galaxy_df = truth_cat_df.merge(
    galaxy_params_df, 
    left_on="cosmodc2_id", 
    right_on="galaxy_id", 
    how="left" 
)
truth_galaxy_psf_df = truth_galaxy_df.merge(
    psf_params_df, 
    left_on="match_objectId", 
    right_on="objectId", 
    how="left" 
)

In [8]:
truth_galaxy_psf_df = truth_galaxy_df.merge(
    psf_params_df, 
    left_on="match_objectId", 
    right_on="objectId", 
    how="left" 
)

In [7]:
merged_catalog_pikle_file = output_dir / "merged_catalog.pkl"
if merged_catalog_pikle_file.exists():
    with open(merged_catalog_pikle_file, "rb") as inputp: 
        truth_galaxy_psf_df = pickle.load(inputp)
else:
    truth_cat_df = pd.DataFrame(truth_cat_data)
    galaxy_params_df = pd.DataFrame(galaxy_params_data)
    psf_params_df = pd.DataFrame(psf_params_data)
    truth_galaxy_df = truth_cat_df.merge(
        galaxy_params_df, 
        left_on="cosmodc2_id", 
        right_on="galaxy_id", 
        how="left" 
    )
    truth_galaxy_psf_df = truth_galaxy_df.merge(
        psf_params_df, 
        left_on="match_objectId", 
        right_on="objectId", 
        how="left" 
    )
    with open(merged_catalog_pikle_file, "wb") as outp:
        pickle.dump(truth_galaxy_psf_df, outp, pickle.HIGHEST_PROTOCOL)

In [11]:
# filter the object by flux
flux_min = 50
truth_galaxy_psf_df = truth_galaxy_psf_df.loc[truth_galaxy_psf_df["flux_r"] > flux_min]

In [13]:
truth_galaxy_psf_df["blendedness"].describe()

count    9.160922e+06
mean     3.976622e-02
std      1.079735e-01
min     -1.678246e+00
25%      0.000000e+00
50%      1.636379e-03
75%      2.584892e-02
max      1.000000e+00
Name: blendedness, dtype: float64

In [14]:
truth_galaxy_psf_df["blendedness"].isna().sum()

3452525

In [15]:
len(truth_galaxy_psf_df)

12613447

In [16]:
truth_galaxy_psf_df["blendedness"] = np.clip(truth_galaxy_psf_df["blendedness"].values, a_min=0.0, a_max=1.0)

In [17]:
truth_galaxy_psf_df["blendedness"].describe()

count    9.160922e+06
mean     4.019095e-02
std      1.076950e-01
min      0.000000e+00
25%      0.000000e+00
50%      1.636379e-03
75%      2.584892e-02
max      1.000000e+00
Name: blendedness, dtype: float64

In [18]:
# # create a pickle file 
truth_galaxy_psf_df.to_pickle(output_dir / f"merged_catalog_with_flux_over_{flux_min}.pkl")

In [ ]:
truth_galaxy_psf_df

# generate split file

In [10]:
print("+" * 100, flush=True)
print("initialization begins", flush=True)

with initialize(config_path=".", version_base=None):
    notebook_cfg = compose("notebook_config")
print("initialization ends", flush=True)
print("+" * 100, flush=True)

print("+" * 100, flush=True)
print("load dc2", flush=True)
dc2: DC2DataModule = instantiate(notebook_cfg.surveys.dc2)
dc2.prepare_data()
print("+" * 100, flush=True)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
initialization begins
initialization ends
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
load dc2


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
